<a href="https://colab.research.google.com/github/mille055/stanford_project/blob/dockerversion/app/notebooks/Stanford_CS231N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
COLAB_FLAG = True   # whether running on colab or locally on computer

In [2]:
if COLAB_FLAG:
  %pip install pydicom==2.1.2
  %pip install monai seaborn sentence_transformers
  !git clone -b dockerversion 'https://github.com/mille055/stanford_project.git'
  

    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=77acda56aebfbe9b683cb590a9bec8e0ad098214a13ebd785fb375eba449f1d7
  Stored in directory: 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from datetime import datetime

from sentence_transformers import SentenceTransformer

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy
from pprint import pprint
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastcore.utils import gt
from fastcore.foundation import L

from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
import sys
import importlib
import warnings
from google.colab import drive
warnings.filterwarnings("ignore")

In [4]:
#local imports
if COLAB_FLAG:
    sys.path.append('/content/stanford_project/app/scripts/')
    train_datafile = '/content/stanford_project/app/data/trainfiles.csv'
    val_datafile = '/content/stanford_project/app/data/valfiles.csv'
    test_datafile = '/content/stanford_project/app/data/testfiles.csv'
    
    drive.mount('/content/gdrive')

else: # running locally
    sys.path.append('/Users/cmm/Documents/GitHub/new/stanford_project/app/scripts/')
    #sys.path.append('/workspaces/stanford_project/app/scripts/')
    train_datafile = '../data/trainfiles.csv'
    val_datafile = '../data/valfiles.csv'
    test_datafile = '../data/testfiles.csv'


### local imports ###
from config import file_dict, feats, feats_to_keep, column_lists, RF_parameters, classes, model_paths
from config import abd_label_dict, val_list, train_val_split_percent, random_seed, data_transforms
from config import sentence_encoder, series_description_column
from utils import *

# from NLP.NLP_training import train_NLP_model
# from NLP.NLP_inference import get_NLP_inference
from cnn.cnn_dataset import ImgDataset
from cnn.cnn_inference import image_to_tensor, pixel_inference, test_pix_model, load_pixel_model, visualize_results
from cnn.cnn_model import *
from cnn.cnn_training import *
from cnn.cnn_data_loaders import get_data_loaders
# from metadata.meta_inference import meta_inference, calc_feature_importances
# from metadata.meta_training import train_fit_parameter_trial, train_meta_model, evaluate_meta_model
# from fusionmodel.fus_model import FusionModel
# from fusionmodel.fus_inference import get_fusion_inference, get_fusion_inference_from_file
# from fusionmodel.fus_training import train_fusion_model, CustomDataset
from model_container import ModelContainer
# from process_tree import Processor


Mounted at /content/gdrive


In [6]:


train_df, val_df, test_df = create_datasets(train_datafile, val_datafile, test_datafile, '/volumes/cm7/Abdominal_MRI_dataset_split/', '/content/gdrive/MyDrive/WW_MRI_abd2/split/' )



In [12]:
# Different combinations to evaluate
models = [CustomResNet50, CustomResNet50b, CustomDenseNet]
optimizers = [optim.SGD(model.parameters(), lr=0.001, momentum=0.9), optim.Adam]
loss_funcs = [nn.CrossEntropyLoss, FocalLoss]



TypeError: ignored

In [17]:
# Get the data loaders
batch_size = 64
train_loader, val_loader, test_loader, dataset_sizes = get_data_loaders(train_df, val_df, test_df, batch_size)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

In [ ]:
# perform several training runs to compare accuracy on the validation dataset

for model_class in models:
        for opt_class in optimizers:
            for loss_func_class in loss_funcs:
                print(f"\nTraining with {model_class.__name__}, {opt_class.__name__}, {loss_func_class.__name__}")

                # Instantiate the custom model
                num_classes = len(classes)
                model = model_class(num_classes)
                model = model.to(device)  # Move the model to the appropriate device

                # Define loss function and optimizer
                criterion = loss_func_class()
                optimizer = opt_class(model.parameters(), lr=0.001, momentum=0.9)

                # Decay LR by a factor of 0.1 every 7 epochs
                exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

                # Train the model
                trained_model, best_val_accuracy, history = train_cnn_model(model, dataloaders, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

                # Plot training curves
                combination_name = f"{model_class.__name__}_{opt_class.__name__}_{loss_func_class.__name__}"
                plot_training_curves(history, combination_name)

                # Evaluate the model on the test dataset
                preds, probs = pixel_inference(test.fname.tolist())
                true = test.true
                accuracy = np.sum(preds==true)/len(true)
                print('Accuracy on the test dataset is ', np.round(accuracy, 3))
                #results = make_results_df(preds, true, test)

                # Save the trained model if needed
                save_filename = f"cnn_model_{combination_name}_{datetime.now().strftime('%Y%m%d')}.pth"
                torch.save(trained_model.state_dict(), save_filename)



Training with CustomResNet50, SGD, CrossEntropyLoss


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 234MB/s]


Epoch 0/24
----------
train Loss: 2.6141 Acc: 0.1841
val Loss: 2.3900 Acc: 0.3277

Epoch 1/24
----------
train Loss: 1.9564 Acc: 0.4850
val Loss: 1.8098 Acc: 0.4887

Epoch 2/24
----------
train Loss: 1.4499 Acc: 0.6267
val Loss: 1.4298 Acc: 0.6299

Epoch 3/24
----------
train Loss: 1.1060 Acc: 0.7180
val Loss: 1.1944 Acc: 0.6751

Epoch 4/24
----------
train Loss: 0.8801 Acc: 0.7845
val Loss: 1.0392 Acc: 0.7316

Epoch 5/24
----------
train Loss: 0.7112 Acc: 0.8232
val Loss: 0.9521 Acc: 0.7232

Epoch 6/24
----------
train Loss: 0.5869 Acc: 0.8612
val Loss: 0.8635 Acc: 0.7571

Epoch 7/24
----------
train Loss: 0.5124 Acc: 0.8663
val Loss: 0.8545 Acc: 0.7684

Epoch 8/24
----------
train Loss: 0.5104 Acc: 0.8714
val Loss: 0.8501 Acc: 0.7655

Epoch 9/24
----------
train Loss: 0.4968 Acc: 0.8766
val Loss: 0.8418 Acc: 0.7684

Epoch 10/24
----------
train Loss: 0.4823 Acc: 0.8897
val Loss: 0.8461 Acc: 0.7684

Epoch 11/24
----------
train Loss: 0.4766 Acc: 0.8912
val Loss: 0.8351 Acc: 0.7712

Ep